In [1]:
from loaders import DLTLoader

In [2]:
d = DLTLoader("logs/dlt/HPA_partnumber not updated.dlt")
msgs = d.get_messages()

In [3]:
num_msg = len(msgs)
num_msg

590698

In [4]:
import re

def sub_keywords(text, extension = []):
    def replace(match):
        return re.sub(r'\w', '?', match.group(0))
    
    keywords = ["HPA", "HIA", "HIB", "hal", "csp", "zen", "IMU", "MAM", "IMMO"]
    keywords.extend(extension)
    pattern = r'(?:' + '|'.join(re.escape(keyword) for keyword in keywords) + r')\S*\b'
    
    return re.sub(pattern, replace, text, flags=re.IGNORECASE)

def sub_ip_address(text):
    def replace(match):
        return re.sub(r'\d', '#', match.group(0))
    
    ip_pattern = r'\b(?:\d{1,3}\.){3}\d{1,3}(?::\d{1,5})?\b'
    return re.sub(ip_pattern, replace, text)


# Example usage
text = "M:MAM1:1061038:Digital Key HIA_asd and ZEN_231. HPA_12 is another keyword. IMUAngularVelocityCompensated hal-srs-monitor! 192.168.1.1:99999 gagagg IMUsad \n \
    electricdriveimmobilisercontrol.1.0.electricdriveimmobilisercontrol_default \n \
    IMMO::1061037:Immobilizer Manager Main::+ \n\
    IMMO:AFW:1061037:Application_framework_core:5:+"
result = sub_ip_address(sub_keywords(text))
print(result)

M:????:???????:??????? Key ??????? and ???????. ?????? is another keyword. ????????????????????????????? ???-???-???????! ###.###.#.#:##### gagagg ?????? 
     electricdrive??????????????????.?.?.??????????????????????????????????????? 
     ????::???????:??????????? Manager Main::+ 
    ????:???:???????:??????????????????????????:?:+


In [17]:
with open('obfuscated/obfuscated_full.txt', 'w') as f:
    for i in range(num_msg):
        msg = msgs[i]
        f.write(sub_ip_address(sub_keywords(str(msg), [msg.apid])) + "\n")

In [6]:
# keep unique lines

unique = set()
with open('obfuscated/obfuscated_compact_full.txt', 'w') as f:
    for i in range(num_msg):
        msg = msgs[i]
        payload = msg.payload_decoded
        if payload in unique:
            continue
        print(sub_ip_address(sub_keywords(str(msg), [msg.apid])), file=f)
        unique.add(payload)

In [4]:
from datetime import datetime

def fmt_time(timestamp):

    dt_object = datetime.fromtimestamp(timestamp)

    return dt_object.strftime('%a %b %d %H:%M:%S %Y')

In [9]:
m = msgs[-1]
for i in dir(m):
    if not i.startswith("__"):
        print(i, " ---  ", m.__getattribute__(i))

_b_base_  ---   None
_b_needsfree_  ---   1
_fields_  ---   [('found_serialheader', <class 'ctypes.c_byte'>), ('resync_offset', <class 'ctypes.c_int'>), ('headersize', <class 'ctypes.c_int'>), ('datasize', <class 'ctypes.c_int'>), ('headerbuffer', <class 'dlt.core.core_base.c_ubyte_Array_42'>), ('databuffer', <class 'dlt.core.core_base.LP_c_ubyte'>), ('databuffersize', <class 'ctypes.c_uint'>), ('p_storageheader', <class 'dlt.core.core_base.LP_cDltStorageHeader'>), ('p_standardheader', <class 'dlt.core.core_base.LP_cDltStandardHeader'>), ('headerextra', <class 'dlt.core.core_base.cDltStandardHeaderExtra'>), ('p_extendedheader', <class 'dlt.core.core_base.LP_cDltExtendedHeader'>)]
_is_extended_header_exists  ---   33
_objects  ---   {'1:5': <ctypes.c_char_Array_88 object at 0x7f9b4c24d540>, '0:5': {}, '7': {'1': <dlt.core.core_base.cDltStorageHeader object at 0x7f9b4c24d740>, '0': {}}, '8': {'1': <dlt.core.core_base.cDltStandardHeader object at 0x7f9b4c24d7c0>, '0': {}}, 'a': {'1': <dlt

In [5]:
m = msgs[0]

In [6]:
str(m)

'Tue Oct 10 12:40:43 2023 921.8996 58 HPA ZQF1 ZQF1 1061101 log warn verbose 1 bazel-out/k8-opt-ST-d9fd4f79f947/bin/deployment/z2_vcc_spa2/service_agents/communication/common/csp/_virtual_includes/csp_property_getter/common/csp_property_getter.h:35: Failed to get property value; group=feature_modes, error code=-1 (kFailure)'

In [7]:
f"{fmt_time(m.storage_timestamp)} {m.tmsp} {m.mcnt} {m.ecuid} {m.apid} {m.ctid} {m.seid} {m.type_string.decode('utf-8')} \
{m.subtype_string.decode('utf-8')} {m.mode_string.decode('utf-8')} {m.is_mode_verbose} {m.payload_decoded}"

'Tue Oct 10 14:40:43 2023 921.8996 58 HPA ZQF1 ZQF1 1061101 log warn verbose 1 bazel-out/k8-opt-ST-d9fd4f79f947/bin/deployment/z2_vcc_spa2/service_agents/communication/common/csp/_virtual_includes/csp_property_getter/common/csp_property_getter.h:35: Failed to get property value; group=feature_modes, error code=-1 (kFailure)'

In [11]:
print(f'''
m.storage_timestamp: {fmt_time(m.storage_timestamp)} OR {m.storage_timestamp} 
m.tmsp: {m.tmsp} 
m.mcnt: {m.mcnt} 
m.ecuid: {m.ecuid} 
m.apid: {m.apid} 
m.ctid: {m.ctid} 
m.seid: {m.seid} 
m.type_string: {m.type_string.decode('utf-8')}
m.subtype_string: {m.subtype_string.decode('utf-8')} 
m.mode_string: {m.mode_string.decode('utf-8')} 
m.is_mode_verbose: {m.is_mode_verbose} 
m.payload_decoded: {m.payload_decoded}''')


m.storage_timestamp: Tue Oct 10 14:40:43 2023 OR 1696941643.669 
m.tmsp: 921.8996 
m.mcnt: 58 
m.ecuid: HPA 
m.apid: ZQF1 
m.ctid: ZQF1 
m.seid: 1061101 
m.type_string: log
m.subtype_string: warn 
m.mode_string: verbose 
m.is_mode_verbose: 1 
m.payload_decoded: bazel-out/k8-opt-ST-d9fd4f79f947/bin/deployment/z2_vcc_spa2/service_agents/communication/common/csp/_virtual_includes/csp_property_getter/common/csp_property_getter.h:35: Failed to get property value; group=feature_modes, error code=-1 (kFailure)
